In [2]:
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
import re
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait




In [5]:
# 수집할 맥주 목록
beer_alist = ['kloud', 'fitz super clear','Asahi super dry', 'Tsingtao', 'Heineken',
 'Kirin ichiban', 'Sapporo Premium Beer', 'stella artois', 'guinness ',
 '1664 Blanc', 'pilsner urquell', 'San Miguel', 'OB premier pilsner', 'cass',
 'stout', 'dry finish', 'max hite', 'hite extra cold', 'victoria bitter', 'BINTANG pilsner',
 'krombacher weizen', 'Miller Genuine Draft', 'Hoegaarden Rosé', 
 "The Premium Malt's", 'REEPER B Weissbier', 'Reeper B. India Pale Ale', 'TIGER BEER',
 'TSINGTAO WHEAT BEER', 'Erdinger Weissbier', 'Carlsberg', 'Budweiser ', 'Hoegaarden Original White Ale',
 'YEBISU', 'Weissbier Naturtrüb', 'Desperados', 'Peroni Nastro Azzurro',
 'Edelweiss Snowfresh', 'Heineken Dark Lager', 'Kozel Dark 10', 'Guinness original',
 'Filite', 'Jeju Wit Ale', 'Stephans Bräu Pilsner', 'Stephans Bräu Premium Lager',
 'Stephans Bräu Hefe-Weizen Naturtrüb', 'Bali Hai Premium', 'Apostel Brau',
 'Egger Zwickl', 'Egger', 'Holsten Premium', 'Franziskaner Premium Weissbier',
 'Egger Radler Grapefruit', 'Bavaria Beer', 'Bavaria 8.6 Original', 'Lapin Kulta IV A',
 'Grolsch Premium Lager', 'Gambrinus Original', 'XXXX Gold', 'Leffe Brown',
 'Lowenbrau Original', 'Asahi Super dray Black ', 'Harbin Beer', "beck's",
 'Hoegaarden Rosee','Platinum Pale Ale', 'Ambar Especial',
 'Schöfferhofer Grapefruit', 'Volfas Engelman Grünberger Hefeweizen', 'Berliner Kindl Pilsener ',
 'BURGE MEESTER', 'Red Rock', 'Erdinger Dunkel', 'Premium German Pilsener', "Queen's Ale Blonde Type"]

# 데이터프레임으로 저장
beer_alist = pd.DataFrame(data=beer_alist, columns=['검색이름'])

In [6]:
beer_alist

,검색이름
0,kloud
1,fitz super clear
2,Asahi super dry
3,Tsingtao
4,Heineken
...,...
69,BURGE MEESTER
70,Red Rock
71,Erdinger Dunkel
72,Premium German Pilsener


In [8]:
# 데이터 프레임 생성
data = pd.DataFrame(data=[], columns=['맥주정보', '검색이름', '맥주이름'])

# chromedriver.exe 파일 경로 설정
chromedriver = 'chromedriver.exe'
# 크롤링 할 경로 설정
url='https://www.beeradvocate.com/'

# 셀레니움으로 웹브라우저를 오픈합니다.
browser = webdriver.Chrome(chromedriver)
browser.get(url)
time.sleep(1)

C:\Users\sun14\AppData\Local\Temp\ipykernel_32084\3496973661.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(chromedriver)


In [5]:
data

,맥주정보,검색이름,맥주이름


In [4]:
def crawl(browser, beer, data, k):
    # 데이터 프레임 생성
    data = pd.DataFrame(data=[], columns=['맥주정보', '검색이름', '맥주이름'])

    # url open
    print('url_open... {0} 맥주 데이터를 수집합니다..'.format(beer))
    browser = webdriver.Chrome("./chromedriver")
    url='https://www.beeradvocate.com/'
    browser.get(url)
    
    # 맥주 검색
    time.sleep(1)
    browser.find_element(By.CSS_SELECTOR,'#QuickSearchQuery').send_keys(beer)
    #맥주검색 클릭
    while 1:
        try:
            browser.find_element(By.CSS_SELECTOR,'#content > div > div > div.uix_contentFix > aside > div > div > div:nth-child(2) > div > form > input.button.primary.Tooltip').click()
            break
        except:
            print('오류발생 다시 검색.')
    time.sleep(1)
    #로그인
    #email = str(input('이메일을 입력하세요 : '))
    #password = str(input('패스워드를 입력하세요 : '))
    #아이디
    element = WebDriverWait(browser, 10).until(EC.visibility_of_any_elements_located((By.ID, "ctrl_pageLogin_login")))[0]
    element.send_keys('asd14987@naver.com')
    #비밀번호
    element_a= WebDriverWait(browser, 10).until(EC.visibility_of_any_elements_located((By.ID, "ctrl_pageLogin_password")))[0]
    element_a.send_keys('1221alsgur')
    #로그인엔터
    browser.find_element(By.CSS_SELECTOR,'#pageLogin > dl.ctrlUnit.submitUnit > dd > input').send_keys(Keys.ENTER)
    #첫번째 맥주선택
    try:
        browser.find_element(By.CSS_SELECTOR,'#ba-content > div:nth-child(3) > div:nth-child(2) > a:nth-child(2)').send_keys(Keys.ENTER)
    except:
        print('바로맥주선택.')
    time.sleep(1)
    #맥주이름수집
    a=browser.find_element(By.XPATH,'//*[@id="content"]/div/div/div[3]/div/div/div[2]/h1')
    beer_name = a.text.split('\n')[0]
    
    error_cnt = 0

    while 1:
        try :
            # 전체 리뷰 개수 수집
            time.sleep(3)
            string= browser.find_element(By.CSS_SELECTOR,'#info_box > div:nth-child(3) > dl > dd:nth-child(18) > span > b').text
            # ,가 포함되어 있는지에 대한 로직
            extract = re.compile('[0-9]*,*[0-9]+')
            str_num = extract.findall(string)
            str_num = str_num[0]

            print('성공... while문을 탈출합니다.')
            break
        except :
            print('오류 발생.. 다시 시작합니다.')

            error_cnt += 1

            if error_cnt == 5:
                print('연속된 오류로 다음 맥주로 넘어갑니다...')
                return

    if ',' in str_num:
        str_num = str_num.split(',')
        str_num = int(str_num[0]+str_num[1])
        num = str_num
    else:
        num = int(str_num)    

    # 수집할 Page 수를 계산합니다.
    page_num = num // 40 + 1

    c=browser.current_url
    id_str = c.split('/')[-3]
    id_2str = c.split('/')[-2]


    time.sleep(1)
    for i in range(page_num):
        print(i+1, '번째 페이지입니다.')
        url = '/beer/profile/'+ id_str+'/'+id_2str+'/?view=beer&sort=&start=' + str(i*40)
        browser.get('https://www.beeradvocate.com' + url)
        time.sleep(3)
        #beer_info=browser.find_elements(By.CSS_SELECTOR,'#rating_fullview_content_2 > span.BAscore_norm').text+" "+browser.find_element(By.CSS_SELECTOR,'#rating_fullview_content_2 > span:nth-child(8)').text+" "+browser.find_element(By.CSS_SELECTOR,'#rating_fullview_content_2 > div').text
        #beer_info=browser.find_element(By.CSS_SELECTOR,'#rating_fullview_container').text
        html = browser.page_source
        soup=BeautifulSoup(html,'html.parser')
        beer_info=soup.select('#rating_fullview_content_2')
        tmp = []
        for info in beer_info:
            text = info.text
            tmp.append(text)
       
        

        # 수집한 것을 데이터프레임에 저장
        #tmp.append(beer_info)
        tmp = pd.DataFrame(data=tmp, columns=['맥주정보'])
        tmp['맥주이름'] = beer_name
        tmp['검색이름'] = beer
        data = pd.concat([data, tmp])
    

    # 데이터가 중복 수집될 경우 리뷰 수 만큼만 Cut
    if num != len(data):
        data = data[:num]

    print('리뷰수 : ', num, '수집된 리뷰수 : ', len(data))

    # 데이터를 csv, excel 파일로 저장합니다.
    result = pd.merge(data, beer_alist, on='검색이름', how='left')
    result.to_csv("beer2_n_"+str(k)+".csv", encoding='utf-8')

    browser.quit()

    return result


In [49]:
# 맥주 리스트 개수만큼 데이터 수집
for k in range(len(beer_alist)):
    result = crawl(browser, beer_alist['검색이름'].iloc[k], data, k)

url_open... Filite 맥주 데이터를 수집합니다..


C:\Users\sun14\AppData\Local\Temp\ipykernel_32084\1198715283.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome("./chromedriver")


바로맥주선택.
성공... while문을 탈출합니다.
1 번째 페이지입니다.
리뷰수 :  6 수집된 리뷰수 :  6


In [51]:
a=pd.read_csv('전처리전데이터2.csv')

In [52]:
a

,Unnamed: 0,맥주정보,검색이름,맥주이름
0,0,Rated: 3.47 by dbrauneis from North CarolinaAp...,kloud,Kloud Original Gravity
1,1,"Rated: 3.75 by etan420 from IllinoisMar 25, 2023",kloud,Kloud Original Gravity
2,2,Rated: 2.95 by Oleg85 from Russian FederationF...,kloud,Kloud Original Gravity
3,3,Rated: 2.79 by Kiro from Russian FederationFeb...,kloud,Kloud Original Gravity
4,4,"Rated: 5 by aloesaba415Jan 09, 2023",kloud,Kloud Original Gravity
...,...,...,...,...
109048,3,Rated: 3.27 by mcthfg from Korea (South)Jun 04...,Queen's Ale Blonde Type,Queen's Ale – Blonde Type
109049,4,Rated: 3.5 by hazelash from Korea (South)Oct 0...,Queen's Ale Blonde Type,Queen's Ale – Blonde Type
109050,5,Reviewed by hoppyhound from Ohio3.63/5 rDev +...,Queen's Ale Blonde Type,Queen's Ale – Blonde Type
109051,6,Reviewed by Fatehunter from Oregon3.44/5 rDev...,Queen's Ale Blonde Type,Queen's Ale – Blonde Type


In [50]:
import pandas as pd

# 합친 데이터를 저장할 데이터프레임
data = pd.DataFrame(data=[], columns=['맥주정보', '검색이름', '맥주이름'])

# 수집한 파일의 개수
files_cnt = 74

for i in range(files_cnt):

    
    try : 
        tmp = pd.read_csv(r'C:\Users\sun14\Desktop\졸작\beer2_n_'+str(i)+'.csv', index_col=0)
        data = pd.concat([data,tmp])
    # 오류 발생 시 넘어갑니다.
    except :
        print(i, '번째에서 오류가 발생했습니다. 다음 파일로 넘어갑니다.')

# 합친 데이터를 저장합니다.
data.to_csv('전처리전데이터2.csv', encoding='utf-8')